In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import sys
import pandas as pd
sys.path.append(r"C:\Users\minkyu\Desktop\K-TACC-main\utils") # 깃헙에서 레포 다운 받은 path 지정 해줘야함 
from BertAugmentation import BertAugmentation


In [ ]:
BERT_aug = BertAugmentation()
random_masking_insertion = BERT_aug.random_masking_insertion

Device set to use cuda:0


In [6]:
df = pd.read_csv('train_df.csv')

In [10]:
filtered_df = df[df['공종(중분류)'] != '철근콘크리트공사']

In [16]:
from tqdm import tqdm
tqdm.pandas()

filtered_df['aug'] = filtered_df['사고원인'].progress_apply(
    lambda sentence: random_masking_insertion(sentence, 0.2) if isinstance(sentence, str) else sentence
)


100%|██████████| 14716/14716 [07:16<00:00, 33.68it/s]
C:\Users\minkyu\AppData\Local\Temp\ipykernel_10044\2282848037.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['aug'] = filtered_df['사고원인'].progress_apply(


In [ ]:
filtered_df

In [42]:
BERT_aug = BertAugmentation()
random_masking_replacement = BERT_aug.random_masking_replacement
random_masking_insertion = BERT_aug.random_masking_insertion

Device set to use cuda:0


In [47]:
import pandas as pd
from tqdm import tqdm

# filtered_df는 원본 DataFrame이라고 가정합니다.
augmented_df_rmr = filtered_df.copy()
augmented_df_rmi = filtered_df.copy()

ratio = 0.2  # 증강 비율

for idx, row in tqdm(filtered_df.iterrows(), total=len(filtered_df), desc="증강 진행"):
    sentence = row['사고원인']
    
    # 문자열이 아닐 경우 문자열로 변환 (예: NaN 또는 float인 경우)
    if not isinstance(sentence, str):
        sentence = str(sentence)
    
    # 두 가지 증강 방법 적용
    augmented_sentence_rmr = random_masking_replacement(sentence, ratio)
    augmented_sentence_rmi = random_masking_insertion(sentence, ratio)
    
    # 증강된 결과를 각 DataFrame에 대체
    augmented_df_rmr.at[idx, '사고원인'] = augmented_sentence_rmr
    augmented_df_rmi.at[idx, '사고원인'] = augmented_sentence_rmi

증강 진행:   0%|          | 0/14716 [00:00<?, ?it/s]

증강 진행: 100%|██████████| 14716/14716 [17:23<00:00, 14.11it/s]


In [54]:
augmented_df_rmr['사고원인'].head()

0    미끄럼 위험이 있는 사다리 답단에 미끄럼방지 장치 설치, 전도 방지조치 등 사다리 ...
1      로프를 이용 한 후 불량으로 묶어둔 배관 서포트가 지면으로 30Cm 가량 협착 발생.
6                                토목공사 경계석설치 작업중 어깨인대손상
7                                           자재 이동중 넘어짐
8                         모래통에 모래를 담고, 이동중 넘어져 왼쪽으로 다침
Name: 사고원인, dtype: object

In [51]:
augmented_df_rmi['사고원인'].head()

0    미끄럼 위험이 항상 있는 사다리 답단에는 미끄럼방지 테이프 설치 등 장애물 전도 방...
1    로프를 잘못 이용만 한 후 양중작업상태 불량으로 묶어 둔 배관 서포트가 지면으로 3...
6                             최근 토목공사 경계석설치 작업중 어깨인대손상
7                                          자재가 이동중 넘어짐
8              모래통에 모래를 가득 담고 짊어지고 이동중 넘어져 왼쪽 팔과 팔을 다침
Name: 사고원인, dtype: object

In [55]:
filtered_df = filtered_df.reset_index(drop=True)
augmented_df_rmr = augmented_df_rmr.reset_index(drop=True)
augmented_df_rmi = augmented_df_rmi.reset_index(drop=True)

In [ ]:
augmented_df_rmr = augmented_df_rmr.drop(columns= ['aug','cosine_similarity','bert_similarity'])
augmented_df_rmi = augmented_df_rmi.drop(columns= ['aug','cosine_similarity','bert_similarity'])

In [67]:
augmented_df_rmr.to_csv("augmented_df_rmr.csv", index=False, encoding='utf-8-sig')
augmented_df_rmi.to_csv("augmented_df_rmi.csv", index=False, encoding='utf-8-sig')


In [2]:
import numpy as np
import pandas as pd

In [9]:
train = pd.read_csv('train_df.csv')
augmented_df_rmr = pd.read_csv("augmented_df_rmr.csv")

In [7]:
len(train)

21069

In [10]:
len(augmented_df_rmr)

14716

In [11]:
combined_df = pd.concat([train, augmented_df_rmr], ignore_index=True)


In [76]:
combined_df2 = pd.concat([combined_df, augmented_df_rmr], ignore_index=True)


In [81]:
combined_df.to_csv("train_rmi.csv", index=False, encoding='utf-8-sig')
combined_df2.to_csv("train_rmi_rmr.csv", index=False, encoding='utf-8-sig')

In [13]:
combined_df.to_csv("train+rmr.csv", index=False, encoding='utf-8-sig')